In [0]:
import numpy as np
import pandas as pd
import keras
import csv

In [73]:
#TEST DATA
traindf = pd.read_csv('testData.csv', delimiter = ',')
traindf = traindf.dropna(axis =1)
traindf.head()

,Sentence,QorA
0,Did his mother die of pneumonia,1
1,Is it derived from the Latin spoken by the rom...,1
2,Why did Bequerel win the Nobel Prize in Physics,1
3,Where is Jakarta located,1
4,"9 screens, but felt that was too much of a sa...",0


In [74]:
import spacy
nlp = spacy.load('en_core_web_md')


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [10]:
#TRAINING DATA
in_txt = open("Toucan/test-inputs.txt").read().splitlines()
document = []
doc = {}
l = []
for line in in_txt:
    l.append(line)
    document.append(nlp(line)) 
doc = {'sent': l}
docdf = pd.DataFrame.from_dict(doc)

In [9]:
#TRAINING DATA TO NLP
s_list = []
for index, row, q in traindf.itertuples():
    traindf['Sentence'][index] = nlp(traindf['Sentence'][index])
    s_list.append(traindf['Sentence'][index]) 

In [77]:
#GET TOKEN TAGS FROM TRAINING DATA
def get_token_tag(df):
    question_df = pd.DataFrame(columns = ['WHpos', 'WHword', 'WHnbor']) #, 'RootToken'])
    for t in df:
        wh_pos = " "
        wh_word = " "
        wh_bi_gram = []
        wh_nbor_pos = " "
        for token in t:
            if token.tag_ == "WDT" or token.tag_ == "WP" or token.tag_ == "WP$" or token.tag_ == "WRB":
                wh_pos = token.tag_
                wh_word = token.text
                if len(t)-1 != token.i:
                    wh_nbor_pos = token.nbor() 
                wh_bi_gram.append(token.text)
                wh_bi_gram.append(str(token.i + 1))
    q_dict = {'WHpos': wh_pos, 'WHword': wh_word, 'WHnbor': wh_nbor_pos} #, 'Root-POS': root_token}
    question_df = question_df.append(q_dict, ignore_index=True)
    q_df = pd.get_dummies(question_df, columns = ['WHpos', 'WHword', 'WHnbor']) #, 'RootToken'])
    return q_df

'\'    if token.dep_ == "ROOT":\n        root_token = token.tag_\n        ttraindf[\'RootToken\'][token.i] = root_token  '

In [7]:
question_train_df = get_token_tag(s_list)

In [8]:
predict_test_df = get_token_tag(document)

In [6]:
from sklearn.svm import LinearSVC
from scipy.sparse import csr_matrix

In [112]:
# CREATING DATA MATRIX
train = traindf['Sentence'].tolist()
predict = docdf['sent'].tolist() 
both_list = list(set(train + predict))
vl = []
training_dict = {}
for val in both_list:
    if val not in traindf:
        training_dict[val] = [0 for i in range(len(traindf.index))]
    else:
        training_dict[val] = list(traindf[val])


"'\nXT_train = pd.DataFrame(training_dict)\nXT_train = csr_matrix(XT_train)\n\n\nXT_predict = pd.DataFrame(predicting_dict)\nXT_predict = csr_matrix(XT_predict)  "

In [2]:
predicting_dict = {}
for col in both_list:
    if col not in docdf:
        predicting_dict[col] = [0]
    else:
        predicting_dict[col] = list(docdf[col])  # KeyError

In [3]:
XT_predict = pd.DataFrame(predicting_dict)
XT_train = pd.DataFrame(training_dict)

In [5]:
#SVC
def classifysvm(df_train, classdf, df_predict):
    lin_clf = LinearSVC()
    lin_clf.fit(df_train, classdf)
    prediction = lin_clf.predict(df_predict)
    return prediction, lin_clf

In [4]:
classdata = traindf['QorA']
finalprediction, svclf = classifysvm(train, classdata, predict)